In [1]:
!ls

README.md		    miniconda.sh  quick_start_pytorch.ipynb   wandb
eliciting-latent-sentiment  miniconda3	  quick_start_pytorch_images


In [2]:
%cd eliciting-latent-sentiment

/notebooks/eliciting-latent-sentiment


In [3]:
#!pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
!pip install fancy_einsum==0.0.3
!pip install transformer_lens
!pip install jaxtyping==0.2.13
!pip install einops
!pip install protobuf==3.20.*
!pip install plotly
!pip install torchtyping
!pip install git+https://github.com/neelnanda-io/neel-plotly.git
!pip install circuitsvis
# !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
# %pip install git+https://github.com/neelnanda-io/PySvelte.git
# %pip install typeguard==2.13.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.9/109.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 124.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.12.0
    Uninstalling huggingface-hub-0.12.0:
      Successfully

In [6]:
from IPython import get_ipython
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import einops
from functools import partial
import torch
import datasets
from torch import Tensor
from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets
from jaxtyping import Float, Int, Bool
from typing import Dict, Iterable, List, Tuple, Union
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_dataset, tokenize_and_concatenate, get_act_name, test_prompt
from transformer_lens.hook_points import HookPoint
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from circuitsvis.activations import text_neuron_activations
from utils.store import load_array, save_html, save_array, is_file, get_model_name, clean_label, save_text
from utils.circuit_analysis import get_logit_diff

In [10]:
def names_filter(name: str):
    """Filter for the names of the activations we want to keep to study the resid stream."""
    return name.endswith('resid_post') or name == get_act_name('resid_pre', 0)

def get_layerwise_token_mean_activations(model: HookedTransformer, data_loader: DataLoader, token_id: int = 13) -> Float[Tensor, "layer d_model"]:
    """Get the mean value of a token across layers"""
    num_layers = model.cfg.n_layers
    d_model = model.cfg.d_model
    
    activation_sums = torch.stack([torch.zeros(d_model) for _ in range(num_layers)]).to(device)
    comma_counts = [0] * num_layers

    print(activation_sums.shape)

    token_mean_values = torch.zeros((num_layers, d_model))
    for _, batch_value in tqdm(enumerate(data_loader), total=len(data_loader)):
        
        batch_tokens = batch_value['tokens'].to(device)

        # get positions of all 11 and 13 token ids in batch
        punct_pos = find_positions(batch_tokens, token_ids=[13])

        _, cache = model.run_with_cache(
            batch_tokens, 
            names_filter=names_filter
        )

        
        for i in range(batch_tokens.shape[0]):
            for p in punct_pos[i][0]:
                for layer in range(num_layers):
                    activation_sums[layer] += cache[f"blocks.{layer}.hook_resid_post"][i, p, :]
                    comma_counts[layer] += 1

    for layer in range(num_layers):
        token_mean_values[layer] = activation_sums[layer] / comma_counts[layer]

    return token_mean_values

In [14]:
def convert_to_tensors(dataset, column_name='tokens'):
    token_buffer = []
    final_batches = []
    
    for batch in dataset:
        trimmed_batch = batch[column_name] #[batch[column_name][0]] + [token for token in batch[column_name] if token != 0]
        final_batches.append(trimmed_batch)
    
    # Convert list of batches to tensors
    final_batches = [torch.tensor(batch, dtype=torch.long) for batch in final_batches]
    # Create a new dataset with specified features
    features = Features({"tokens": Sequence(Value("int64"))})
    final_dataset = Dataset.from_dict({"tokens": final_batches}, features=features)

    final_dataset.set_format(type="torch", columns=["tokens"])
    
    return final_dataset

## Comma Ablation on Natural Text

In [16]:
torch.set_grad_enabled(False)
device = "cuda"
MODEL_NAME = "EleutherAI/pythia-1.4b"
model = HookedTransformer.from_pretrained(
    MODEL_NAME,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=False,
    device=device,
)
model.name = MODEL_NAME

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-1.4b into HookedTransformer


In [17]:
from datasets import load_from_disk
# this loads the SST dataset as produced by the treebank.py script
sst_data = load_from_disk("sst2") 
sst_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7864
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2058
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1007
    })
})

In [18]:
sst_data['train'][0]['text'], sst_data['train'][0]['label']

("The Rock is destined to be the 21st Century's new ''Conan'' and that he's going to make a splash even greater than Arnold Schwarzenegger, Jean-Claud Van Damme or Steven Segal.",
 1)

In [19]:
def filter_function(example):
    """Filters the examples based on whether there is a positive logit difference and whether the top answer token is in the top 10 logits"""
    prompt = model.to_tokens(example['text'] + " Review Sentiment:", prepend_bos=False)
    answer = torch.tensor([29071, 32725]).unsqueeze(0).unsqueeze(0).to(device) if example['label'] == 1 else torch.tensor([32725, 29071]).unsqueeze(0).unsqueeze(0).to(device)
    logits, cache = model.run_with_cache(prompt)
    logit_diff = get_logit_diff(logits, answer)
    
    # Determine if the top answer (index 0) token is in top 10 logits
    _, top_indices = logits.topk(10, dim=-1)  # Get indices of top 10 logits
    top_answer_token = answer[0, 0, 0]  # Assuming answer is of shape (1, 1, 2) and the top answer token is at index 0
    is_top_answer_in_top_10_logits = (top_indices == top_answer_token).any()
    
    # Add a new field 'keep_example' to the example
    example['keep_example'] = (logit_diff > 0.0) and is_top_answer_in_top_10_logits.item()
    return example

# Use the map function to apply the filter_function
sst_data_with_flag_train = sst_data['train'].map(filter_function, keep_in_memory=True)
sst_data_with_flag_val = sst_data['dev'].map(filter_function, keep_in_memory=True)
sst_data_with_flag_test = sst_data['test'].map(filter_function, keep_in_memory=True)
sst_data_with_flag = concatenate_datasets([sst_data_with_flag_train, sst_data_with_flag_val, sst_data_with_flag_test])

# Use the filter function to keep only the examples where 'keep_example' is True
sst_zero_shot = sst_data_with_flag.filter(lambda x: x['keep_example'])

# save dataset
sst_zero_shot.save_to_disk("sst_zero_shot")


Map:   0%|          | 0/7864 [00:00<?, ? examples/s]

Map:   0%|          | 0/1007 [00:00<?, ? examples/s]

Map:   0%|          | 0/2058 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10929 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1066 [00:00<?, ? examples/s]

In [20]:
sst_zero_shot

Dataset({
    features: ['text', 'label', 'keep_example'],
    num_rows: 1066
})

In [23]:
from datasets import load_from_disk
sst_zero_shot = load_from_disk("sst_zero_shot")

In [24]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer


# Define the batch size
BATCH_SIZE = 5

# Load a tokenizer (you'll need to specify the appropriate model)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1.4b")
# set padding token
tokenizer.pad_token = 1

#dataset = text_dataset.map(lambda x: tokenize_and_concatenate(x, tokenizer))

def concatenate_classification_prompts(examples):
    return {"text": (examples['text'] + " Review Sentiment:")}


def tokenize_function(examples):
    # Tokenizes and handles padding/truncation of all texts in the dataset
    # Also creates attention masks and labels
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)


def convert_answers(example, pos_answer_id=29071, neg_answer_id=32725):
    if example['label'] == 1:
        answers = torch.tensor([pos_answer_id, neg_answer_id])
    else:
        answers = torch.tensor([neg_answer_id, pos_answer_id])

    return {'answers': answers}


dataset = sst_zero_shot.map(concatenate_classification_prompts, batched=False)
dataset = dataset.map(tokenize_function, batched=False)
dataset = dataset.map(convert_answers, batched=False)
dataset = dataset.rename_column("input_ids", "tokens")
dataset.set_format(type="torch", columns=["tokens", "attention_mask", "label", "answers"])
dataset


Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'keep_example', 'tokens', 'attention_mask', 'answers'],
    num_rows: 1066
})

In [25]:
model.to_string(dataset[0]['tokens']), model.to_str_tokens(dataset[0]['answers'][0])

('Yet the act is still charming here. Review Sentiment:1111111111111111111111111111111111111111111111111111',
 [' Positive'])

In [26]:
from utils.circuit_analysis import get_logit_diff
logits, cache = model.run_with_cache(dataset['tokens'][0])
get_logit_diff(logits, dataset['answers'][0].unsqueeze(0).unsqueeze(0).to(device))

tensor(2.3774, device='cuda:0')

In [27]:
# create a subset with only positive labels
pos_dataset = dataset.filter(lambda example: example['label']==1)
neg_dataset = dataset.filter(lambda example: example['label']==0)
len(pos_dataset), len(neg_dataset)

Filter:   0%|          | 0/1066 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1066 [00:00<?, ? examples/s]

(785, 281)

In [28]:
import random
from datasets import concatenate_datasets

def get_random_subset(dataset, n):
    total_size = len(dataset)
    random_indices = random.sample(range(total_size), n)
    return dataset.select(random_indices)

pos_subset = get_random_subset(pos_dataset, 280)
neg_subset = get_random_subset(neg_dataset, 280)
balanced_subset = concatenate_datasets([pos_subset, neg_subset])
# randomize the order of balanced_subset
balanced_subset = balanced_subset.shuffle(len(balanced_subset))

# Create a new dataloader from the subset, converting the data to tensors
pos_data_loader = DataLoader(
    pos_subset, batch_size=5, shuffle=False, drop_last=True
)
neg_data_loader = DataLoader(
    neg_subset, batch_size=5, shuffle=False, drop_last=True
)
balanced_data_loader = DataLoader(
    balanced_subset, batch_size=5, shuffle=False, drop_last=True
)

# save datasets
pos_subset.save_to_disk("sst_pos_subset")
neg_subset.save_to_disk("sst_neg_subset")
balanced_subset.save_to_disk("sst_balanced_subset")

len(pos_data_loader), len(neg_data_loader)


Saving the dataset (0/1 shards):   0%|          | 0/280 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/280 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/560 [00:00<?, ? examples/s]

(56, 56)

In [30]:
def compute_last_position_logit_diff(logits, mask, answer):
    """
    Parameters:
    - logits: Tensor of shape (batch, sequence_position, logits)
    - mask: Tensor of shape (batch, sequence_position)
    - answer: Tensor of shape (batch, 2)

    Returns:
    - logit_diff: Tensor of shape (batch,)
    """
    # Find the last unmasked sequence position for each item in the batch
    last_unmasked_positions = mask.sum(dim=1) - 1  # Subtract 1 to get zero-based index
    #print(last_unmasked_positions)

    # Extract the logits for the last unmasked positions
    last_logits = logits[torch.arange(logits.size(0)), last_unmasked_positions]
    #print(f"last logits: {last_logits.shape}")
    #print(f"last logits shape: {last_logits.shape}")

    # Extract the logits for the correct and incorrect answers
    correct_logits = last_logits[torch.arange(last_logits.size(0)), answer[:, 0]]
    #print(f"correct logits shape: {correct_logits.shape}")
    incorrect_logits = last_logits[torch.arange(last_logits.size(0)), answer[:, 1]]

    # Compute the logit differences
    logit_diff = correct_logits - incorrect_logits
    #print(f"logit diff shape: {logit_diff.shape}")

    return logit_diff


### Apply Directional Ablation

In [46]:
def load_directions(model, direction_folder):
    directions = []
    for i in range(model.cfg.n_layers):
        dir = np.load(f"{direction_folder}/das_simple_train_ADJ_layer{i}.npy")
        if len(dir.shape) == 2:
            dir = dir[:, 0]
        directions.append(torch.tensor(dir))

    # convert to tensor
    directions = torch.stack(directions).to(device)

    return directions

def get_random_directions(model, num_layers):
    directions = []
    for i in range(num_layers):
        dir = torch.randn(model.cfg.d_model).to(device)
        directions.append(dir)

    # convert to tensor
    directions = torch.stack(directions).to(device)

    return directions

def get_zeroed_dir_vector(model, directions):
    zeroed_directions = []
    for i in range(model.cfg.n_layers):
        dir = torch.zeros(model.cfg.d_model).to(device)
        zeroed_directions.append(dir)

    # convert to tensor
    zeroed_directions = torch.stack(zeroed_directions).to(device)

    return zeroed_directions

#directions = load_directions(model, "data/pythia-2.8b-das")
random_directions = get_random_directions(model, model.cfg.n_layers)
#zeroed_directions = get_zeroed_dir_vector(model, directions)

In [32]:
def ablate_resid_with_direction(
    component: torch.Tensor,
    hook: HookPoint,
    direction_vector: torch.Tensor,
    labels: torch.Tensor,
    multiplier: float = 1.0,
    pos_by_batch: torch.Tensor = None,
    layer: int = 0,
) -> torch.Tensor:
    """
    Ablates a batch tensor by removing the influence of a direction vector from it.

    Args:
        component: the tensor to compute the mean over the batch dim of
        direction_vector: the direction vector to remove from the component
        multiplier: the multiplier to apply to the direction vector
        pos_by_batch: the positions to ablate
        layer: the layer to ablate

    Returns:
        the ablated component
    """
    assert 'resid' in hook.name

    # Normalize the direction vector to make sure it's a unit vector
    D_normalized = direction_vector[layer] / torch.norm(direction_vector[layer])

    # Calculate the projection of component onto direction_vector
    proj = einops.einsum(component, D_normalized, "b s d, d -> b s").unsqueeze(-1) * D_normalized
    

    # Ablate the direction from component
    component_ablated = component.clone()  # Create a copy to ensure original is not modified
    if pos_by_batch is not None:
        batch_indices, sequence_positions = torch.where(pos_by_batch == 1)
        component_ablated[batch_indices, sequence_positions] = component[batch_indices, sequence_positions] - multiplier * proj[batch_indices, sequence_positions]
        
        # Print the (batch, pos) coordinates of all d_model vectors that were ablated
        # for b, s in zip(batch_indices, sequence_positions):
        #     print(f"(batch, pos) = ({b.item()}, {s.item()})")

        # Check that positions not in (batch_indices, sequence_positions) were not ablated
        check_mask = torch.ones_like(component, dtype=torch.bool)
        check_mask[batch_indices, sequence_positions] = 0
        if not torch.all(component[check_mask] == component_ablated[check_mask]):
            raise ValueError("Positions outside of specified (batch_indices, sequence_positions) were ablated!")

    return component_ablated

#### Complete Ablation

In [39]:
def compute_directional_ablation_modified_logit_diff_all_pos(model: HookedTransformer, data_loader: DataLoader, direction_vectors, multiplier=1.0) -> float:
    
    orig_ld_list = []
    ablated_ld_list = []
    freeze_ablated_ld_list = []
    
    for _, batch_value in tqdm(enumerate(data_loader), total=len(data_loader)):
        batch_tokens = batch_value['tokens'].to(device)
        labels = batch_value['label'].to(device)
        all_pos = batch_value['attention_mask'].to(device)

        # get the logit diff for the last token in each sequence
        orig_logits, clean_cache = model.run_with_cache(batch_tokens, return_type="logits", prepend_bos=False)
        orig_ld = compute_last_position_logit_diff(orig_logits, batch_value['attention_mask'], batch_value['answers'])
        orig_ld_list.append(orig_ld)
        
        # repeat with commas ablated
        for layer in layers_to_ablate:
            dir_ablate_comma = partial(ablate_resid_with_direction, labels=labels, direction_vector=direction_vectors, multiplier=multiplier, pos_by_batch=all_pos, layer=layer)
            model.blocks[layer].hook_resid_post.add_hook(dir_ablate_comma)
       
        ablated_logits = model(batch_tokens, return_type="logits", prepend_bos=False)
        # check to see if ablated_logits has any nan values
        if torch.isnan(ablated_logits).any():
            print("ablated logits has nan values")
        ablated_ld = compute_last_position_logit_diff(ablated_logits, batch_value['attention_mask'], batch_value['answers'])
        ablated_ld_list.append(ablated_ld)
        
        model.reset_hooks()

    return torch.cat(orig_ld_list), torch.cat(ablated_ld_list)

In [37]:
model.reset_hooks()

##### Balanced Results

In [47]:
from utils.ablation import freeze_attn_pattern_hook
heads_to_ablate = [(layer, head) for layer in range(model.cfg.n_layers) for head in range(model.cfg.n_heads)]
heads_to_freeze = [(layer, head) for layer in range(model.cfg.n_layers) for head in range(model.cfg.n_heads)]
layers_to_ablate = [layer for layer in range(model.cfg.n_layers)]
#orig_ld_list, ablated_ld_list, freeze_ablated_ld_list = compute_directional_ablation_modified_logit_diff_all_pos(model, balanced_data_loader, directions, 1.0, 13)
orig_ld_list_rand, ablated_ld_list_rand = compute_directional_ablation_modified_logit_diff_all_pos(model, balanced_data_loader, random_directions, 1.0, 13)

#orig_accuracy = (orig_ld_list > 0).float().mean()
#ablated_accuracy = (ablated_ld_list > 0).float().mean()

orig_accuracy_rand = (orig_ld_list_rand > 0).float().mean()
ablated_accuracy_rand = (ablated_ld_list_rand > 0).float().mean()


# print(f"Original mean logit diff: {orig_ld_list.mean():.4f}")
# print(f"Original accuracy: {orig_accuracy:.4f}")
# print("\n")
# print(f"Comma-ablated mean logit diff: {ablated_ld_list.mean():.4f}")
# print(f"Comma-ablated accuracy: {ablated_accuracy:.4f}")
# print(f"Percent drop in logit diff with comma ablation: {(orig_ld_list.mean() - ablated_ld_list.mean()) / orig_ld_list.mean() * 100:.2f}%")
# print(f"Percent drop in accuracy with comma ablation: {(orig_accuracy - ablated_accuracy) / orig_accuracy * 100:.2f}%")
# print("\n")
print("---------------------------------------------------------")
print(f"Original mean logit diff: {orig_ld_list_rand.mean():.4f}")
print("Random direction ablation results:")
print(f"All-pos-ablated mean logit diff: {ablated_ld_list_rand.mean():.4f}")
print(f"Percent drop in logit diff with comma ablation: {(orig_ld_list_rand.mean() - ablated_ld_list_rand.mean()) / orig_ld_list_rand.mean() * 100:.2f}%")
print(f"Original accuracy: {orig_accuracy_rand:.4f}")
print(f"All-pos-ablated accuracy: {ablated_accuracy_rand:.4f}")
print(f"Percent drop in accuracy with comma ablation: {(orig_accuracy_rand - ablated_accuracy_rand) / orig_accuracy_rand * 100:.2f}%")

  0%|          | 0/112 [00:00<?, ?it/s]

---------------------------------------------------------
Original mean logit diff: 1.2261
Random direction ablation results:
All-pos-ablated mean logit diff: 1.2777
Percent drop in logit diff with comma ablation: -4.21%
Original accuracy: 1.0000
All-pos-ablated accuracy: 0.9286
Percent drop in accuracy with comma ablation: 7.14%
